In [1]:
import torch
import torch.nn as nn
from pathlib import Path
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from math import *
from random import gauss,seed
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt


In [2]:
from PythonModules.RESNET import RESNET 

In [3]:
trajdict = np.load('output.npz')
import ast
#params = ast.literal_eval(str(trajdict['params']))
traj_closed_train = trajdict['traj_closed_train_hungarian']
traj_open_train = trajdict['traj_open_train_hungarian']
traj_closed_test = trajdict['traj_closed_test_hungarian']
traj_open_test = trajdict['traj_open_test_hungarian']
x = np.vstack([traj_closed_train, traj_open_train])
xval = np.vstack([traj_closed_test, traj_open_test])

In [4]:
batch_size = 100
train_set = np.vstack([traj_open_train[1000:81000:2], traj_closed_train[1000:81000:2] ] )

In [5]:
inputData = torch.from_numpy(train_set[:70000,:4]).type('torch.FloatTensor').to('cuda')
dataSet32 = torch.from_numpy( np.load('DataSet32.npy')[:70000] ).type('torch.FloatTensor').to('cuda')

In [6]:
dataSet32 = dataSet32.unsqueeze(1)

In [ ]:
temp = 36*torch.softmax( torch.reshape(dataSet32,(dataSet32.shape[0],32*32)), 1)
temp = torch.reshape(temp,(temp.shape[0],1, 32,32))

In [7]:
newDataSet = []

In [8]:
for eachSample in dataSet32:
    diff = 36 - eachSample.sum(axis=1).sum(axis=1)
    tp = eachSample + (eachSample*diff)/eachSample.sum(axis=1).sum(axis=1)
    newDataSet.append(tp.to('cpu').numpy())

In [9]:
dataSet32 = dataSet32.type('torch.DoubleTensor').to('cuda')

In [10]:
dataSet32 = torch.from_numpy(np.array(newDataSet)).type('torch.FloatTensor').to('cuda')

In [11]:
print(dataSet32.type())

torch.cuda.FloatTensor


In [12]:
dataSet32 = dataSet32.to('cpu').type('torch.DoubleTensor').to('cuda')

In [13]:
totalLoss =0.0

In [14]:
train_loader = torch.utils.data.DataLoader(inputData.type('torch.DoubleTensor').to('cuda'), batch_size=batch_size)

In [15]:
testInputData = torch.from_numpy(train_set[70000:,:4]).type('torch.DoubleTensor').to('cuda')
testDataSet32 = torch.from_numpy( np.load('DataSet32.npy')[70000:] ).type('torch.DoubleTensor').to('cuda').unsqueeze(1)

In [16]:
test_loader = torch.utils.data.DataLoader(testInputData.to('cuda'), batch_size=batch_size)

In [17]:
class DensityEstimator(nn.Module):
    def __init__(self,dimer_atoms,output_dim):
        super(DensityEstimator, self).__init__()
        self.dimer_atoms = dimer_atoms
        self.hidden_dim = 16
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.fc1 = nn.Linear(self.dimer_atoms, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.hidden_dim*4)
        self.fc3 = nn.Linear(self.hidden_dim*4, self.hidden_dim*4*4)
        #self.lstm = nn.LSTM(hidden_dim, output_dim)
        #self.fc3 = nn.Linear(self.hidden_dim*4*4, hidden_dim*4)
        self.fc4 = nn.Linear(self.hidden_dim*4*4, output_dim)
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        #out,_ = self.lstm(out.view(self.boxes,1,-1))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out) )
        out = self.relu(self.fc4(out))
        out = 36*torch.softmax(out,1)
        return out

In [18]:
class combineModel(nn.Module):
    def __init__(self,model1,model2,model3):
        super(combineModel,self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
    
    def forward(self,inp):
        out = self.model1(inp)
        out = torch.reshape(out,(out.shape[0],1,8,8))
        #print(out)
        out = self.model2(out)
        out = 36*torch.softmax( torch.reshape(out,(out.shape[0],16*16)), 1)
        print(out.shape[0])
        out = torch.reshape(out,(out.shape[0],1,16,16))
        
        #print(out)
        out = self.model3(out)
        #out = 36*torch.softmax( torch.reshape(out,(out.shape[0],32*32)), 1)
        #out = torch.reshape(out,(out.shape[0],1,32,32))
        
        return out

In [19]:
model1 = DensityEstimator(4,64)
model2 = RESNET()
model3 = RESNET()

In [20]:
model1.load_state_dict(torch.load("ParticleTo8x8"))
model1.eval()

DensityEstimator(
  (relu): LeakyReLU(negative_slope=0.2, inplace=True)
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=64, bias=True)
)

In [21]:
model2.load_state_dict(torch.load("DenseEstimResNet8-16"))
model2.eval()

RESNET(
  (conv_input): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (relu): LeakyReLU(negative_slope=0.2, inplace=True)
  (residual): Sequential(
    (0): Residual_Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Residual_Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): Residual_Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [22]:
model3.load_state_dict(torch.load("DenseEstimResNet"))
model3.eval()

RESNET(
  (conv_input): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (relu): LeakyReLU(negative_slope=0.2, inplace=True)
  (residual): Sequential(
    (0): Residual_Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Residual_Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): Residual_Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): LeakyReLU(negative_slope=0.2, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [23]:
CombineModel = combineModel(model1,model2,model3).type('torch.DoubleTensor').to('cuda')

In [24]:
adOptimizer = optim.Adam(CombineModel.parameters(),lr =0.0000000001)

In [25]:
Epochs = 100

In [ ]:
tl = []

In [ ]:
torch.save(CombineModel.state_dict(), 'DoubleCombineDensityEstimator')

In [ ]:
for epoch in range(Epochs):
    totalLoss = 0.0
    print(epoch)
    if((epoch+1)%2 == 0):
        torch.save(CombineModel.state_dict(), 'DoubleCombineDensityEstimator')
    for i,data in enumerate(train_loader,1):
        adOptimizer.zero_grad()
        out = CombineModel(data)
        #print(out)
        #print(dataSet32[70000+(i-1)*batch_size:70000+i*batch_size])
        loss = torch.sqrt( nn.MSELoss()(out, dataSet32[(i-1)*batch_size:i*batch_size]) )
        loss.backward()
        print(i,loss.item())
        totalLoss += loss.item()
        if(math.isnan(loss.item()) or math.isinf(loss.item())):
            break
        adOptimizer.step()
    if(math.isnan(totalLoss) or math.isinf(totalLoss)):
            break
    tl.append(totalLoss)

In [26]:
CombineModel.load_state_dict(torch.load("DoubleCombineDensityEstimator"))

<All keys matched successfully>

In [ ]:
CombineModel = CombineModel.to('cpu').type('torch.DoubleTensor').to('cuda')

In [27]:
totalLoss =0.0

In [34]:
for i,data in enumerate(train_loader,1):
        print(data.shape)
        out = CombineModel(data)
        loss = torch.sqrt( nn.MSELoss()(out, dataSet32[(i-1)*batch_size:i*batch_size]) )
        print(loss)
        loss =  (out- dataSet32[(i-1)*batch_size:i*batch_size])/dataSet32[(i-1)*batch_size:i*batch_size]
        loss = loss.squeeze()
        loss = torch.reshape(loss,(loss.shape[0],32*32))
        #print(loss.shape)
        loss = torch.abs(loss)
        loss =loss.mean()
        print(loss)
        #loss.backward()
        #print(out[:,0].shape)
        #loss =  (out- dataSet32[(i-1)*batch_size:i*batch_size])/dataSet32[(i-1)*batch_size:i*batch_size]
        #print(loss.mean())
        

torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0394, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0428, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0461, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0454, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0465, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>

tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0412, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0392, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0419, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0364, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0413, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0020, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0507, device='

tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0393, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0402, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0408, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0015, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0355, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0391, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0014, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0335, device='

tensor(0.0012, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0299, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0014, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0330, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0014, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0339, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0013, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0321, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0013, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0317, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0012, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0287, device='

tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0396, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0412, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0418, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0015, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0373, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0421, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0445, device='

...

tensor(0.0020, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0498, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0021, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0551, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0022, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0564, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0023, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0581, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0021, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0529, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0021, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0531, device='

tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0471, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0019, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0494, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0465, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0442, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0464, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0457, device='

tensor(0.0028, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0717, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0030, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0762, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0030, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0765, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0029, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0735, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0028, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0734, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0030, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0778, device='

tensor(0.0017, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0433, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0016, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0405, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0018, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0439, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0020, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0488, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0020, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0513, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0021, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0521, device='

tensor(0.0009, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0197, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0007, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0172, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0007, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0158, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0006, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0151, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0006, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0153, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
torch.Size([100, 4])
100
tensor(0.0006, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward>)
tensor(0.0138, device='

In [ ]:
plt.plot(tl)

In [ ]:
print(tl)

In [ ]:
print(tl)